# EV Charging Station Data Integration and Cleaning
## Introduction
**Electric vehicles (EVs)** are becoming increasingly popular worldwide, necessitating a robust network of charging stations. To facilitate data analysis and planning around EV infrastructure, it's essential to have accurate and comprehensive data on charging stations. This notebook demonstrates the process of acquiring, merging, and cleaning data on EV charging stations from two different sources:

**OpenChargeMap API:** This API provides detailed information about EV charging stations globally, including station locations, connection types, operators, and more.

**Overpass API (OpenStreetMap):** This API allows access to location-based data, including the locations of EV charging stations. It pulls data directly from OpenStreetMap, a community-driven mapping project.

The objective of this notebook is to create a unified dataset that combines data from these two sources, cleans it, and prepares it for analysis. This dataset can then be used for various analytical purposes, such as mapping charger distribution, analyzing charger types and availability, and forecasting demand for charging infrastructure.

## Data Acquisition

In this section, we will fetch data from both the OpenChargeMap and Overpass APIs.


*   OpenChargeMap API: This API provides detailed information about charging stations, including geographical location, connection types, and operator information. We use this API to get a comprehensive dataset of charging stations in Australia.
*   Overpass API: OpenStreetMap (OSM) provides an extensive set of geographical data, including the location of EV charging stations. Using the Overpass API, we can query OSM for all nodes tagged as "charging stations" within Australia.



These two APIs provide complementary data about EV charging stations. By combining them, we can fill in gaps and cross-verify the information for better accuracy and completeness.

## Data Cleaning and Merging

After acquiring data from both sources, we proceed with data cleaning and merging:

1.   Cleaning OpenChargeMap Data: The data from the OpenChargeMap API includes a lot of metadata and redundant information that we don't need for our analysis. In this step, we drop irrelevant columns to focus only on the essential data, such as location, operator, connection type, and cost.
2.   Cleaning Overpass Data: Similar to the OpenChargeMap data, the Overpass data contains extraneous details. We drop irrelevant columns and retain only the useful information, such as geographical coordinates and tags related to the charging station.
1.   Merging Datasets: After cleaning both datasets, we merge them into a single DataFrame. This involves several steps:
Column Renaming: To ensure consistency and avoid conflicts during the merge, we rename columns that represent the same information differently in each dataset (e.g., latitude and longitude).
Unifying Data: We combine columns with similar data (e.g., operator information from both datasets) into unified columns.
Handling Duplicates: We identify and handle duplicate entries based on location coordinates, preferring data from the OpenChargeMap where possible for consistency.











## Data Processing and Transformation
Once the data is merged, we process and transform it to ensure consistency and usability:

Extracting and Normalizing Data: For example, we extract details from the 'connections' column, which contains nested information about the types of connections available at each charging station, such as connection type, power output, and current type.

Handling Missing Values: We address missing data by either filling in default values or marking entries as 'Unknown' where appropriate. This ensures that our dataset is complete and consistent, even if some data is missing.

Converting Data Types: We convert columns to appropriate data types (e.g., numerical, categorical) to facilitate easier analysis and ensure correct calculations during analysis.

Final Data Cleaning: We further refine the dataset by dropping unnecessary columns and combining related information (e.g., merging brand and operator information into a single column).

## Final Dataset Preparation
After all cleaning and processing steps, we finalize the dataset:


*   Standardizing Columns: We rename columns for clarity and consistency (e.g., renaming 'NumberOfPoints' to 'ChargingPoints').
*   Ensuring Data Completeness: We fill any remaining missing values, ensuring the dataset is ready for analysis.
*   Adding Flags and Indicators: We add flags, such as a 'ChargingPoints_Flag' to indicate whether a charging station has zero points and convert it to one to ensure all entries have at least one charging point.


The final dataset is a cleaned, unified, and comprehensive view of EV charging stations in Australia, ready for analysis.

In [2]:
pip install geopy


  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e5/15/cf2a69ade4b194aa524ac75112d5caac37414b20a3a03e6865dfe0bd1539/geopy-2.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for geographiclib<3,>=1.52 from https://files.pythonhosted.org/packages/9f/5a/a26132406f1f40cf51ea349a5f11b0a46cec02a2031ff82e391c2537247a/geographiclib-2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 1.4 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [14]:
import requests
import pandas as pd
import ast
from geopy.distance import geodesic
from scipy.spatial import KDTree
import numpy as np

def get_cleaned_charging_station_data(api_key):
    # Fetch data from Open Charge Map API
    def get_all_chargers_in_au(api_key):
        url = "https://api.openchargemap.io/v3/poi/"
        headers = {'X-API-Key': api_key}
        params = {'countrycode': 'AU', 'maxresults': 10000}
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            response.raise_for_status()

    # Normalize JSON data to DataFrame
    def chargers_to_dataframe(charger_data):
        df = pd.json_normalize(charger_data)
        return df

    # Fetch data from Overpass API
    def get_osm_chargers():
        overpass_url = "http://overpass-api.de/api/interpreter"
        overpass_query = """
        [out:json];
        area["ISO3166-1"="AU"][admin_level=2];
        node["amenity"="charging_station"](area);
        out body;
        """
        response = requests.get(overpass_url, params={'data': overpass_query})
        if response.status_code == 200:
            data = response.json()
            elements = data['elements']
            charging_stations = []
            for element in elements:
                lat = element.get('lat')
                lon = element.get('lon')
                tags = element.get('tags', {})
                station_name = tags.get('name', 'Unknown')
                charging_stations.append({
                    'name': station_name,
                    'latitude': lat,
                    'longitude': lon,
                    'tags': tags
                })
            return pd.DataFrame(charging_stations)
        else:
            raise Exception(f"Error fetching OSM data: {response.status_code}")

    # Extract tags into separate columns
    def extract_tags(df, column_name):
        tags_df = pd.json_normalize(df[column_name])
        df = pd.concat([df.drop(columns=[column_name]), tags_df], axis=1)
        return df

    # Unify common columns into single columns
    def unify_columns(df, col1, col2, unified_col):
        if col1 in df.columns and col2 in df.columns:
            df[unified_col] = df[col1].combine_first(df[col2])
            df.drop([col1, col2], axis=1, inplace=True)
        elif col1 in df.columns:
            df.rename(columns={col1: unified_col}, inplace=True)
        elif col2 in df.columns:
            df.rename(columns={col2: unified_col}, inplace=True)

    # Parse connections
    def parse_connections(connections):
        connection_type = None
        power = None
        current_type = None

        if isinstance(connections, str):
            try:
                connections = ast.literal_eval(connections)
            except (ValueError, SyntaxError):
                connections = []

        if isinstance(connections, list):
            for connection in connections:
                if isinstance(connection, dict):
                    conn_type = connection.get('ConnectionType', {})
                    curr_type = connection.get('CurrentType', {})

                    if isinstance(conn_type, dict) and connection_type is None:
                        connection_type = conn_type.get('Title', 'Unknown')

                    if power is None:
                        power = connection.get('PowerKW', None)

                    if isinstance(curr_type, dict) and current_type is None:
                        current_type = curr_type.get('Title', 'Unknown')

                    break

        return pd.Series({'ConnectionType': connection_type, 'Power': power, 'CurrentType': current_type})

    # Mark duplicates based on rounded coordinates, preferring df_ocm entries
    def mark_prefer_ocm_duplicates(df):
        df['is_duplicate'] = False
        grouped = df.groupby(['lat_round', 'lon_round'])
        for _, group in grouped:
            if len(group) > 1:
                ocm_index = group[group['origin'] == 'ocm'].index
                if not ocm_index.empty:
                    df.loc[group.index.difference(ocm_index), 'is_duplicate'] = True
                else:
                    df.loc[group.index[1:], 'is_duplicate'] = True

    # Fetch OCM data
    charger_data = get_all_chargers_in_au(api_key)
    charger_df = chargers_to_dataframe(charger_data)

    # Drop unnecessary columns from OCM data
    columns_to_drop = [
        'UserComments', 'PercentageSimilarity', 'MediaItems', 'ParentChargePointID',
        'DataProvidersReference', 'OperatorsReference', 'GeneralComments', 'DatePlanned',
        'DateLastConfirmed', 'MetadataValues', 'DataProvider.WebsiteURL',
        'DataProvider.Comments', 'DataProvider.DataProviderStatusType.IsProviderEnabled',
        'DataProvider.DataProviderStatusType.ID', 'DataProvider.DataProviderStatusType.Title',
        'DataProvider.IsRestrictedEdit', 'DataProvider.IsOpenDataLicensed',
        'DataProvider.IsApprovedImport', 'DataProvider.License',
        'DataProvider.DateLastImported', 'DataProvider.ID', 'OperatorInfo.WebsiteURL',
        'OperatorInfo.Comments', 'OperatorInfo.PhonePrimaryContact',
        'OperatorInfo.PhoneSecondaryContact', 'OperatorInfo.IsPrivateIndividual',
        'OperatorInfo.AddressInfo', 'OperatorInfo.BookingURL',
        'OperatorInfo.ContactEmail', 'OperatorInfo.FaultReportEmail',
        'OperatorInfo.IsRestrictedEdit', 'OperatorInfo.ID', 'OperatorInfo', 'UsageType'
    ]
    charger_df = charger_df.drop(columns=columns_to_drop, errors='ignore')

    # Fetch OSM data
    df1 = get_osm_chargers()
    df1 = extract_tags(df1, 'tags')

    # Drop unnecessary columns from OSM data
    columns_to_drop_osm = [
        'opening_hours', 'brand:wikidata', 'operator:wikidata', 'brand:wikipedia',
        'operator:wikipedia', 'description', 'name', 'short_name', 'source', 'location',
        'addr:unit', 'website', 'note', 'ref:ocm', 'motorcar', 'bicycle', 'scooter',
        'access:note', 'alt_name', 'temporary:access', 'source:operator', 'fixme', 'ref',
        'maxstay', 'voltage', 'manufacturer', 'operator:website', 'no:network', 'bus',
        'hgv', 'motorcycle', 'amperage', 'not:brand:wikidata', 'construction', 'ele',
        'not:operator:wikidata', 'man_made', 'phone:AU', 'office', 'phone', 'start_date',
        'wikidata', 'survey:date', 'parking', 'url', 'network:wikidata', 'check_date',
        'indoor', 'branch'
    ]
    df1 = df1.drop(columns=columns_to_drop_osm, errors='ignore')

    # Rename and merge columns
    df_osm = df1.copy()
    df_ocm = charger_df.copy()

    df_osm['origin'] = 'osm'
    df_ocm['origin'] = 'ocm'

    common_columns_osm = {
        'latitude': 'latitude_osm',
        'longitude': 'longitude_osm',
        'amenity': 'amenity_osm',
        'brand': 'brand_osm',
        'operator': 'operator_osm',
        'capacity': 'capacity_osm',
        'fee': 'fee_osm',
        'charge': 'cost_osm',
        'socket:type1': 'socket_type1_osm',
        'socket:type2': 'socket_type2_osm',
        'socket:chademo': 'socket_chademo_osm',
        'addr:housenumber': 'address_house_number_osm',
        'addr:street': 'address_street_osm',
        'addr:city': 'address_city_osm',
        'addr:country': 'address_country_osm',
        'addr:postcode': 'address_postcode_osm',
        'addr:state': 'address_state_osm',
        'website': 'website_osm',
        'network': 'network_osm'
    }

    common_columns_ocm = {
        'AddressInfo.Latitude': 'latitude_ocm',
        'AddressInfo.Longitude': 'longitude_ocm',
        'Connections': 'connections_ocm',
        'UsageCost': 'cost_ocm',
        'OperatorInfo.Title': 'operator_ocm',
        'AddressInfo.Title': 'address_title_ocm',
        'AddressInfo.AddressLine1': 'address_line1_ocm',
        'AddressInfo.AddressLine2': 'address_line2_ocm',
        'AddressInfo.Town': 'address_city_ocm',
        'AddressInfo.StateOrProvince': 'address_state_ocm',
        'AddressInfo.Postcode': 'address_postcode_ocm',
        'AddressInfo.Country.Title': 'address_country_ocm',
        'AddressInfo.ContactTelephone1': 'contact_telephone1_ocm',
        'AddressInfo.ContactTelephone2': 'contact_telephone2_ocm',
        'AddressInfo.ContactEmail': 'contact_email_ocm',
        'AddressInfo.RelatedURL': 'website_ocm'
    }

    df_osm.rename(columns=common_columns_osm, inplace=True)
    df_ocm.rename(columns=common_columns_ocm, inplace=True)

    # Merge DataFrames
    merged_df = pd.concat([df_osm, df_ocm], ignore_index=True, sort=False)

    # Unify common columns
    common_columns = {
        'latitude': ['latitude_osm', 'latitude_ocm'],
        'longitude': ['longitude_osm', 'longitude_ocm'],
        'amenity': ['amenity_osm', 'amenity_ocm'],
        'brand': ['brand_osm', 'brand_ocm'],
        'operator': ['operator_osm', 'operator_ocm'],
        'capacity': ['capacity_osm', 'capacity_ocm'],
        'cost': ['cost_osm', 'cost_ocm'],
        'socket_type1': ['socket_type1_osm', 'socket_type1_ocm'],
        'socket_type2': ['socket_type2_osm', 'socket_type2_ocm'],
        'socket_chademo': ['socket_chademo_osm', 'socket_chademo_ocm'],
        'address_house_number': ['address_house_number_osm', 'address_house_number_ocm'],
        'address_street': ['address_street_osm', 'address_street_ocm'],
        'address_city': ['address_city_osm', 'address_city_ocm'],
        'address_country': ['address_country_osm', 'address_country_ocm'],
        'address_postcode': ['address_postcode_osm', 'address_postcode_ocm'],
        'address_state': ['address_state_osm', 'address_state_ocm'],
        'website': ['website_osm', 'website_ocm'],
        'network': ['network_osm', 'network_ocm']
    }

    for unified_col, cols in common_columns.items():
        unify_columns(merged_df, cols[0], cols[1], unified_col)

    # Round coordinates
    merged_df['lat_round'] = merged_df['latitude'].round(3)
    merged_df['lon_round'] = merged_df['longitude'].round(3)

    # Mark duplicates
    mark_prefer_ocm_duplicates(merged_df)

    # Keep only unique entries
    unique_merged_df = merged_df[merged_df['is_duplicate'] == False]
    unique_merged_df.drop(columns=['is_duplicate', 'origin', 'lat_round', 'lon_round'], inplace=True)

    # List of columns to keep
    columns_to_keep = [
        'brand', 'socket_type1', 'access', 'capacity', 'socket:tesla', 'socket_type2',
        'socket:tesla_supercharger', 'socket:tesla_supercharger:output', 'socket:tesla_supercharger_ccs',
        'socket:tesla_supercharger_ccs:output', 'authentication:app', 'authentication:nfc', 'socket_chademo',
        'socket:chademo:output', 'socket:type2:output', 'socket:type2_combo', 'socket:type2_combo:output',
        'socket:type1:output', 'authentication:none', 'output', 'charging_station:output', 'cost',
        'payment:app', 'socket:tesla_destination', 'socket:tesla_standard', 'socket:tesla_standard:output',
        'authentication:membership_card', 'socket:tesla_destination:output', 'payment:credit_cards',
        'socket:type2_cable', 'payment:contactless', 'socket:type1_combo', 'socket:type1_combo:output',
        'payment:free', 'socket:type2:power', 'socket:j1772', 'socket:type2:current', 'socket:type2:voltage',
        'payment:tap_to_pay', 'socket:type2_cable:current', 'socket:type2_cable:output',
        'socket:type2_cable:voltage', 'payment:via_app', 'socket:wall', 'payment:cash',
        'payment:coins', 'payment:mastercard', 'payment:visa', 'payment:debit_cards',
        'payment:cards', 'payment:membership_card', 'payment:qr_code', 'socket:unknown',
        'payment:apple_pay', 'socket:ccs:output', 'ID', 'connections_ocm', 'NumberOfPoints',
        'UsageType.IsPayAtLocation', 'UsageType.IsMembershipRequired', 'UsageType.IsAccessKeyRequired',
        'StatusType.IsOperational', 'StatusType.IsUserSelectable', 'latitude', 'longitude',
        'operator', 'fee_osm'
    ]
    df_cleaned = unique_merged_df[columns_to_keep]

    # Parse connections
    df_cleaned[['ConnectionType', 'Power', 'CurrentType']] = df_cleaned['connections_ocm'].apply(parse_connections)

    # Ensure numeric conversion
    df_cleaned['Power'] = pd.to_numeric(df_cleaned['Power'], errors='coerce')
    df_cleaned['output'] = pd.to_numeric(df_cleaned['output'], errors='coerce')

    # Merge 'Power' and 'output' columns
    df_cleaned['Combined_Power_Output'] = df_cleaned.apply(
        lambda row: max(filter(pd.notnull, [row['Power'], row['output']]), default=None), axis=1
    )

    # Drop unnecessary columns
    columns_to_drop = [col for col in df_cleaned.columns if col.startswith('payment:') or col.startswith('socket:') or col.startswith('authentication:')]
    df_cleaned.drop(columns=columns_to_drop, inplace=True)

    # Combine 'brand' and 'operator' into a single 'operator' column
    def combine_brand_operator(row):
        if pd.notnull(row['brand']):
            return row['brand']
        elif pd.notnull(row['operator']):
            return row['operator']
        return None

    df_cleaned['operator'] = df_cleaned.apply(combine_brand_operator, axis=1)

    # Ensure numeric conversion for 'capacity' and 'NumberOfPoints'
    df_cleaned['capacity'] = pd.to_numeric(df_cleaned['capacity'], errors='coerce')
    df_cleaned['NumberOfPoints'] = pd.to_numeric(df_cleaned['NumberOfPoints'], errors='coerce')

    # Add 'capacity' to 'NumberOfPoints' and drop 'capacity'
    df_cleaned['NumberOfPoints'] = df_cleaned['NumberOfPoints'].fillna(0) + df_cleaned['capacity'].fillna(0)
    df_cleaned.drop(columns=['capacity'], inplace=True)

    # Drop specified columns
    columns_to_drop = ['brand', 'access', 'output', 'charging_station:output',
                       'connections_ocm', 'ID', 'StatusType.IsUserSelectable',
                       'fee_osm', 'Combined_Power_Output']
    df_cleaned.drop(columns=columns_to_drop, inplace=True)

    # Replace values in socket columns only if they already have a value
    df_cleaned['socket_type1'] = df_cleaned['socket_type1'].apply(lambda x: 'Type 1 (J1772)' if pd.notna(x) else x)
    df_cleaned['socket_type2'] = df_cleaned['socket_type2'].apply(lambda x: 'Type 2 (Socket Only)' if pd.notna(x) else x)
    df_cleaned['socket_chademo'] = df_cleaned['socket_chademo'].apply(lambda x: 'CHAdeMO' if pd.notna(x) else x)

    # Merge connection types
    def merge_connection_types(row):
        if pd.notna(row['ConnectionType']):
            return row['ConnectionType']
        sockets = [row['socket_type1'], row['socket_type2'], row['socket_chademo']]
        sockets = list(filter(pd.notna, sockets))
        return ', '.join(sockets) if sockets else row['ConnectionType']

    df_cleaned['ConnectionType'] = df_cleaned.apply(merge_connection_types, axis=1)
    df_cleaned.drop(columns=['socket_type1', 'socket_type2', 'socket_chademo'], inplace=True)

    # Rename columns for clarity
    df_cleaned.rename(columns={
        'NumberOfPoints': 'ChargingPoints',
        'UsageType.IsPayAtLocation': 'PayAtLocation',
        'UsageType.IsMembershipRequired': 'MembershipRequired',
        'UsageType.IsAccessKeyRequired': 'AccessKeyRequired',
        'StatusType.IsOperational': 'IsOperational',
        'latitude': 'Latitude',
        'longitude': 'Longitude',
        'operator': 'Operator',
        'ConnectionType': 'ConnectionType',
        'Power': 'PowerOutput',
        'CurrentType': 'CurrentType',
        'cost': 'Cost'
    }, inplace=True)

    # Fill missing 'Cost' and 'CurrentType' with 'Unknown'
    df_cleaned['Cost'].fillna('Unknown', inplace=True)
    df_cleaned['CurrentType'].fillna('Unknown', inplace=True)

    df_cleaned['CurrentType'].fillna('Unknown', inplace=True)
    df_cleaned['PayAtLocation'].fillna('Unknown', inplace=True)
    df_cleaned['MembershipRequired'].fillna('Unknown', inplace=True)
    df_cleaned['AccessKeyRequired'].fillna('Unknown', inplace=True)
    df_cleaned['IsOperational'].fillna('Unknown', inplace=True)
    df_cleaned['Operator'].fillna('Unknown', inplace=True)
    df_cleaned['ConnectionType'].fillna('Unknown', inplace=True)

    # Handle missing 'PowerOutput' by filling with np.nan
    df_cleaned['PowerOutput'].fillna(np.nan, inplace=True)

    # Convert any 0 values in 'ChargingPoints' to 1 and add a flag column
    df_cleaned['ChargingPoints_Flag'] = df_cleaned['ChargingPoints'].apply(lambda x: 1 if x == 0 else 0)
    df_cleaned['ChargingPoints'] = df_cleaned['ChargingPoints'].apply(lambda x: 1 if x == 0 else x)

    # Ensure data types are correct
    df_cleaned['ChargingPoints'] = df_cleaned['ChargingPoints'].astype(int)
    df_cleaned['IsOperational'] = df_cleaned['IsOperational'].astype('category')
    df_cleaned['ConnectionType'] = df_cleaned['ConnectionType'].astype('category')
    df_cleaned['Operator'] = df_cleaned['Operator'].astype('category')
    df_cleaned['CurrentType'] = df_cleaned['CurrentType'].astype('category')
    df_cleaned['Cost'] = df_cleaned['Cost'].astype('category')
    df_cleaned['PowerOutput'] = pd.to_numeric(df_cleaned['PowerOutput'], errors='coerce')
    df_cleaned['IsOperational'] = df_cleaned['IsOperational'].astype('category')
    df_cleaned['ConnectionType'] = df_cleaned['ConnectionType'].astype('category')
    df_cleaned['Operator'] = df_cleaned['Operator'].astype('category')
    df_cleaned['PayAtLocation'] = df_cleaned['PayAtLocation'].astype('category')
    df_cleaned['MembershipRequired'] = df_cleaned['MembershipRequired'].astype('category')
    df_cleaned['AccessKeyRequired'] = df_cleaned['AccessKeyRequired'].astype('category')

    return df_cleaned

# Modified main execution part
API_KEY = "38d18ea7-9248-46cf-860e-2a915c9b172e"

# Get cleaned data
cleaned_df = get_cleaned_charging_station_data(API_KEY)

# Save to CSV
csv_filename = "cleaned_charging_stations.csv"
cleaned_df.to_csv(csv_filename, index=False)
print(f"\nData saved to {csv_filename}")
print(f"Number of records saved: {len(cleaned_df)}")

# Display sample data
print("\nFirst few records of the saved data:")
print(cleaned_df.head())



Data saved to cleaned_charging_stations.csv
Number of records saved: 1712

First few records of the saved data:
      Cost  ChargingPoints PayAtLocation MembershipRequired AccessKeyRequired  \
0  Unknown               1       Unknown            Unknown           Unknown   
2  Unknown               1       Unknown            Unknown           Unknown   
3  Unknown               2       Unknown            Unknown           Unknown   
4  Unknown               8       Unknown            Unknown           Unknown   
8  Unknown               4       Unknown            Unknown           Unknown   

  IsOperational   Latitude   Longitude            Operator  \
0       Unknown -31.936252  115.871531             Unknown   
2       Unknown -32.935880  151.643848         ChargePoint   
3       Unknown -41.218297  146.412523             Unknown   
4       Unknown -34.755480  149.720994  Tesla Supercharger   
8       Unknown -37.887228  145.082491         Tesla, Inc.   

         ConnectionType  Po

/var/folders/xc/kb91yk9x24942px3_wqcy2s00000gn/T/ipykernel_58328/241522816.py:241: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_merged_df.drop(columns=['is_duplicate', 'origin', 'lat_round', 'lon_round'], inplace=True)
/var/folders/xc/kb91yk9x24942px3_wqcy2s00000gn/T/ipykernel_58328/241522816.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[['ConnectionType', 'Power', 'CurrentType']] = df_cleaned['connections_ocm'].apply(parse_connections)
/var/folders/xc/kb91yk9x24942px3_wqcy2s00000gn/T/ipykernel_58328/241522816.py:266: SettingWithCopyWar

In [10]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1712 entries, 0 to 2126
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   Cost                 1712 non-null   category
 1   ChargingPoints       1712 non-null   int64   
 2   PayAtLocation        1712 non-null   category
 3   MembershipRequired   1712 non-null   category
 4   AccessKeyRequired    1712 non-null   category
 5   IsOperational        1712 non-null   category
 6   Latitude             1712 non-null   float64 
 7   Longitude            1712 non-null   float64 
 8   Operator             1712 non-null   category
 9   ConnectionType       1712 non-null   category
 10  PowerOutput          1117 non-null   float64 
 11  CurrentType          1712 non-null   category
 12  ChargingPoints_Flag  1712 non-null   int64   
dtypes: category(8), float64(3), int64(2)
memory usage: 110.1 KB


In [15]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from timezonefinder import TimezoneFinder

class MelbourneEVDataEnricher:
    def __init__(self, df):
        """Initialize with existing Melbourne charging station DataFrame"""
        self.df = df.copy()
        self.tf = TimezoneFinder()
        
    def add_location_based_data(self):
        """Add Melbourne-specific location-based enrichment data"""
        # Melbourne population density varies by suburb (people per km²)
        # CBD: ~20,000, Inner suburbs: 5,000-8,000, Outer suburbs: 2,000-4,000
        self.df['PopulationDensity'] = np.where(
            (self.df['Latitude'].between(-37.81, -37.82)) & 
            (self.df['Longitude'].between(144.95, 144.97)),
            np.random.uniform(15000, 20000, size=len(self.df)),  # CBD
            np.where(
                (self.df['Latitude'].between(-37.75, -37.85)) & 
                (self.df['Longitude'].between(144.90, 145.00)),
                np.random.uniform(5000, 8000, size=len(self.df)),  # Inner suburbs
                np.random.uniform(2000, 4000, size=len(self.df))   # Outer suburbs
            )
        )
        
        # Distance to major roads (Monash Freeway, Western Ring Road, etc.)
        self.df['DistanceToHighway'] = np.random.uniform(0.1, 3.0, size=len(self.df))
        
        # Daily traffic volume based on location
        self.df['DailyTrafficVolume'] = np.where(
            (self.df['Latitude'].between(-37.81, -37.82)) & 
            (self.df['Longitude'].between(144.95, 144.97)),
            np.random.randint(20000, 50000, size=len(self.df)),  # CBD
            np.random.randint(5000, 20000, size=len(self.df))    # Other areas
        )
        
        # Nearby amenities (Melbourne-specific)
        self.df['NearbyRestaurants'] = np.where(
            (self.df['Latitude'].between(-37.81, -37.82)) & 
            (self.df['Longitude'].between(144.95, 144.97)),
            np.random.randint(10, 30, size=len(self.df)),  # CBD
            np.random.randint(2, 10, size=len(self.df))    # Other areas
        )
        
        # Shopping centers and parking
        self.df['NearbyShoppingCenters'] = np.random.randint(0, 3, size=len(self.df))
        self.df['ParkingCapacity'] = np.where(
            (self.df['Latitude'].between(-37.81, -37.82)) & 
            (self.df['Longitude'].between(144.95, 144.97)),
            np.random.randint(20, 100, size=len(self.df)),  # CBD
            np.random.randint(5, 20, size=len(self.df))     # Other areas
        )
        
        # Land use types (Melbourne-specific)
        melbourne_land_uses = [
            'Commercial', 'Residential', 'Mixed Use', 
            'Activity Centre', 'Industrial', 'Public Use'
        ]
        self.df['LandUseType'] = np.random.choice(melbourne_land_uses, size=len(self.df))
        
        # Median income by area (based on Melbourne demographics)
        self.df['MedianIncome'] = np.where(
            (self.df['Latitude'].between(-37.81, -37.82)) & 
            (self.df['Longitude'].between(144.95, 144.97)),
            np.random.uniform(85000, 120000, size=len(self.df)),  # CBD and inner suburbs
            np.random.uniform(65000, 85000, size=len(self.df))    # Other areas
        )
        
        # Melbourne climate zones
        self.df['ClimateZone'] = 'Temperate Oceanic (Cfb)'  # Melbourne's Köppen climate classification
        
        return self

    def add_time_usage_data(self):
        """Add Melbourne-specific time and usage patterns"""
        # Melbourne peak hours
        def generate_melbourne_peak_hours():
            morning_start = np.random.randint(7, 9)  # Melbourne morning peak
            evening_start = np.random.randint(16, 18)  # Melbourne evening peak
            return f"{morning_start:02d}:00-{morning_start+2:02d}:00, {evening_start:02d}:00-{evening_start+2:02d}:00"
        
        self.df['PeakHours'] = [generate_melbourne_peak_hours() for _ in range(len(self.df))]
        
        # Usage patterns (adjusted for Melbourne)
        self.df['AvgChargingDuration'] = np.random.uniform(30, 90, size=len(self.df))  # minutes
        self.df['DailyUtilizationRate'] = np.where(
            (self.df['Latitude'].between(-37.81, -37.82)) & 
            (self.df['Longitude'].between(144.95, 144.97)),
            np.random.uniform(0.5, 0.9, size=len(self.df)),  # CBD
            np.random.uniform(0.3, 0.7, size=len(self.df))   # Other areas
        )
        
        # Peak days (accounting for Melbourne's business districts)
        self.df['WeeklyPeakDay'] = np.random.choice(
            ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],
            size=len(self.df),
            p=[0.2, 0.2, 0.25, 0.25, 0.1]  # Higher mid-week usage
        )
        
        # Station age (based on Melbourne's EV infrastructure timeline)
        installation_dates = pd.date_range(start='2019-01-01', end='2023-12-31', periods=len(self.df))
        self.df['InstallationDate'] = np.random.choice(installation_dates, size=len(self.df))
        
        return self

    def add_vehicle_data(self):
        """Add Melbourne-specific vehicle data"""
        # Popular EV models in Melbourne/Australia
        melbourne_ev_models = [
            'Tesla Model 3', 'Tesla Model Y', 
            'BYD Atto 3', 'MG ZS EV', 
            'Hyundai Kona Electric', 'Polestar 2'
        ]
        
        model_probabilities = [0.3, 0.2, 0.15, 0.15, 0.1, 0.1]  # Based on Australian EV sales
        self.df['MostCommonEVModel'] = np.random.choice(
            melbourne_ev_models, 
            size=len(self.df),
            p=model_probabilities
        )
        
        # Battery capacities based on common Australian models
        self.df['AvgBatteryCapacity'] = np.random.uniform(60, 85, size=len(self.df))  # kWh
        
        # Charging speeds available in Melbourne
        self.df['MaxChargingSpeed'] = np.where(
            self.df['ConnectionType'] == 'CCS (Type 2)',
            np.random.uniform(150, 250, size=len(self.df)),
            np.where(
                self.df['ConnectionType'] == 'CHAdeMO',
                np.random.uniform(50, 100, size=len(self.df)),
                np.random.uniform(7, 22, size=len(self.df))
            )
        )
        
        return self

    def add_all_enrichments(self):
        """Add all Melbourne-specific enrichment data"""
        return self.add_location_based_data().add_time_usage_data().add_vehicle_data()

    def get_enriched_dataframe(self):
        """Return the enriched DataFrame"""
        return self.df

def enrich_melbourne_charging_data(input_df):
    """Main function to enrich Melbourne charging station data"""
    enricher = MelbourneEVDataEnricher(input_df)
    enriched_df = enricher.add_all_enrichments().get_enriched_dataframe()
    return enriched_df

if __name__ == "__main__":
    try:
        # Load the cleaned data
        original_df = pd.read_csv("cleaned_charging_stations.csv")
        print(f"Loaded {len(original_df)} records from cleaned data")
        
        # Enrich the data
        enriched_df = enrich_melbourne_charging_data(original_df)
        
        # Save enriched data
        enriched_filename = "melbourne_enriched_charging_stations.csv"
        enriched_df.to_csv(enriched_filename, index=False)
        print(f"\nEnriched data saved to {enriched_filename}")
        print(f"Total records: {len(enriched_df)}")
        
        # Display sample of new columns
        new_columns = set(enriched_df.columns) - set(original_df.columns)
        print("\nNew columns added:")
        for col in sorted(new_columns):
            print(f"- {col}")
            
        # Display sample of enriched data
        print("\nSample of enriched data (first record):")
        sample_record = enriched_df.iloc[0]
        for col in new_columns:
            print(f"{col}: {sample_record[col]}")
            
    except Exception as e:
        print(f"Error in data enrichment process: {e}")

Loaded 1712 records from cleaned data

Enriched data saved to melbourne_enriched_charging_stations.csv
Total records: 1712

New columns added:
- AvgBatteryCapacity
- AvgChargingDuration
- ClimateZone
- DailyTrafficVolume
- DailyUtilizationRate
- DistanceToHighway
- InstallationDate
- LandUseType
- MaxChargingSpeed
- MedianIncome
- MostCommonEVModel
- NearbyRestaurants
- NearbyShoppingCenters
- ParkingCapacity
- PeakHours
- PopulationDensity
- WeeklyPeakDay

Sample of enriched data (first record):
DailyTrafficVolume: 6072
ParkingCapacity: 11
MostCommonEVModel: MG ZS EV
DailyUtilizationRate: 0.31041454815244207
ClimateZone: Temperate Oceanic (Cfb)
PopulationDensity: 2090.3347791444667
AvgChargingDuration: 44.54398862887143
NearbyRestaurants: 5
PeakHours: 07:00-09:00, 17:00-19:00
AvgBatteryCapacity: 72.75224032329703
DistanceToHighway: 2.9693471795305757
WeeklyPeakDay: Monday
MedianIncome: 77297.18360039391
InstallationDate: 2019-08-18 07:47:56.212741088
LandUseType: Activity Centre
MaxCh